<a href="https://colab.research.google.com/github/ankitgurjarr/summer-training/blob/main/day10/Assignment/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_json('/content/Sarcasm_Headlines_Dataset_v2.json',lines = True)

In [ ]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
###Drop Nan Values
df=df.dropna()

df.isnull().sum()

is_sarcastic    0
headline        0
article_link    0
dtype: int64

In [ ]:
## Get the Independent Features

X=df.drop(['article_link','is_sarcastic'],axis=1)

In [ ]:
## Get the Dependent features
y=df['is_sarcastic']

In [ ]:
X.shape

(28619, 1)

In [ ]:
y.shape

(28619,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages['headline'][1]

'dem rep. totally nails why congress is falling short on gender, racial equality'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
len(messages)

28619

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['headline'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

    
    

Streaming output truncated to the last 5000 lines.
23619
23620
23621
23622
23623
23624
23625
23626
23627
23628
23629
23630
23631
23632
23633
23634
23635
23636
23637
23638
23639
23640
23641
23642
23643
23644
23645
23646
23647
23648
23649
23650
23651
23652
23653
23654
23655
23656
23657
23658
23659
23660
23661
23662
23663
23664
23665
23666
23667
23668
23669
23670
23671
23672
23673
23674
23675
23676
23677
23678
23679
23680
23681
23682
23683
23684
23685
23686
23687
23688
23689
23690
23691
23692
23693
23694
23695
23696
23697
23698
23699
23700
23701
23702
23703
23704
23705
23706
23707
23708
23709
23710
23711
23712
23713
23714
23715
23716
23717
23718
23719
23720
23721
23722
23723
23724
23725
23726
23727
23728
23729
23730
23731
23732
23733
23734
23735
23736
23737
23738
23739
23740
23741
23742
23743
23744
23745
23746
23747
23748
23749
23750
23751
23752
23753
23754
23755
23756
23757
23758
23759
23760
23761
23762
23763
23764
23765
23766
23767
23768
23769
23770
23771
23772
23773
23774
23775
23776
2

In [ ]:
corpus

['thirtysometh scientist unveil doomsday clock hair loss',
 'dem rep total nail congress fall short gender racial equal',
 'eat veggi delici differ recip',
 'inclement weather prevent liar get work',
 'mother come pretti close use word stream correctli',
 'white inherit',
 'way file tax less stress',
 'richard branson global warm donat nearli much cost fail balloon trip',
 'shadow govern get larg meet marriott confer room b',
 'lot parent know scenario',
 'lesbian consid father indiana amaz one',
 'amanda peet told daughter sex special hug',
 'know regard current treatment ebola',
 'chri christi suggest hillari clinton blame boko haram kidnap hundr schoolgirl',
 'ford develop new suv run pure gasolin',
 'uber ceo travi kalanick step trump econom advisori council',
 'area boy enter jump touch top doorway phase',
 'area man travel gurney',
 'leav person disabl behind',
 'lin manuel miranda would like remind put phone away',
 'journalist kill target intern press rise',
 'guard video game 

**Embedding Representation**

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3030, 4223, 3976, 230, 292, 4858, 410],
 [2631, 2015, 2280, 2137, 1879, 4187, 3329, 790, 4152, 4931],
 [4607, 959, 2903, 4563, 3300],
 [1980, 4197, 4603, 1545, 1515, 1958],
 [2264, 4624, 524, 4336, 1046, 1785, 2199, 1598],
 [397, 1304],
 [2021, 3307, 4839, 3396, 3575],
 [707, 2598, 319, 3301, 3887, 3636, 4576, 3918, 595, 3726, 2487],
 [1489, 2585, 1515, 1461, 4674, 2087, 3587, 3936, 4448],
 [2903, 981, 3735, 1020],
 [723, 695, 574, 905, 1168, 3089],
 [1275, 1074, 1380, 888, 599, 4904, 1571],
 [3735, 3950, 2238, 3328, 1654],
 [3099, 3986, 1834, 2780, 145, 3903, 3766, 1588, 2545, 3785, 174],
 [4358, 2724, 1166, 2128, 3318, 2977, 2382],
 [2780, 4725, 4148, 4732, 3257, 4145, 761, 852, 4417],
 [2232, 4599, 1941, 3832, 2096, 3906, 4804, 4291],
 [2232, 1897, 4220, 2893],
 [29, 1856, 3217, 1794],
 [1836, 2339, 3261, 4198, 4870, 2998, 805, 8, 2854],
 [4883, 1538, 3371, 4993, 2783, 1682],
 [1217, 4943, 4521, 337, 682, 2103, 4461, 4880, 214],
 [4728],
 [4520, 3700, 4423, 3071, 1992],
 [1268, 14

In [ ]:

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  292 4858  410]
 [   0    0    0 ...  790 4152 4931]
 [   0    0    0 ... 2903 4563 3300]
 ...
 [   0    0    0 ... 4741  918 2311]
 [   0    0    0 ... 1818 2306 1559]
 [   0    0    0 ... 2271 2774 4247]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 3030, 4223, 3976,  230,  292, 4858,  410], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(28619, (28619,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((28619, 20), (28619,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
300/300 [==============================] - 14s 38ms/step - loss: 0.5632 - accuracy: 0.6992 - val_loss: 0.5025 - val_accuracy: 0.7538
Epoch 2/10
300/300 [==============================] - 11s 36ms/step - loss: 0.4277 - accuracy: 0.8025 - val_loss: 0.5065 - val_accuracy: 0.7534
Epoch 3/10
300/300 [==============================] - 11s 36ms/step - loss: 0.3664 - accuracy: 0.8410 - val_loss: 0.5135 - val_accuracy: 0.7571
Epoch 4/10
300/300 [==============================] - 11s 36ms/step - loss: 0.3174 - accuracy: 0.8668 - val_loss: 0.5383 - val_accuracy: 0.7517
Epoch 5/10
300/300 [==============================] - 12s 39ms/step - loss: 0.2804 - accuracy: 0.8854 - val_loss: 0.6433 - val_accuracy: 0.7458
Epoch 6/10
300/300 [==============================] - 11s 36ms/step - loss: 0.2539 - accuracy: 0.8976 - val_loss: 0.7053 - val_accuracy: 0.7441
Epoch 7/10
300/300 [==============================] - 11s 36ms/step - loss: 0.2295 - accuracy: 0.9099 - val_loss: 0.7169 - val_accuracy:

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
predict_x=model.predict(X_test) 
y_pred=np.argmax(predict_x,axis=1)


In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)


array([[4916,    0],
       [4529,    0]])

In [ ]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5204870301746956

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5204870301746956